In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras_tuner as kt

In [2]:
data=pd.read_csv('Alphabets_data.csv')

In [3]:
# data Normalization
from sklearn.preprocessing import StandardScaler

# Assuming the last column is the target
X = data.drop(columns='letter')
y = data['letter']

# Normalizing features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


In [4]:
# encoding target columns as the target we have is object data type
from sklearn.preprocessing import LabelEncoder
# Convert labels to numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.3, random_state=42)


In [6]:

# Define the model creation function for Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=16, max_value=64, step=16),
                    activation=hp.Choice('activation', values=['relu', 'tanh']),
                    input_shape=(X_train.shape[1],)))
    for _ in range(hp.Int('num_layers', 1, 3)):  # Number of hidden layers
        model.add(Dense(units=hp.Int('units', min_value=16, max_value=64, step=16),
                        activation=hp.Choice('activation', values=['relu', 'tanh'])))
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Initialize the Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='intro_to_kt'
)




C:\Users\shrut\anaconda3\envs\priyanka_code\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)



In [8]:
# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[early_stopping])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]


Trial 60 Complete [00h 00m 14s]
val_accuracy: 0.8014285564422607

Best val_accuracy So Far: 0.9350000023841858
Total elapsed time: 00h 07m 27s


In [9]:
# Evaluate the best model
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc}")

# Generate predictions
y_pred_proba = best_model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)  # Convert probabilities to class labels


Test Accuracy: 0.9326666593551636
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [10]:



# Print classification report
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
# Print precision, recall, and F1 score
print("Precision Score:", precision_score(y_test, y_pred, average='weighted'))
print("Recall Score:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))



Classification Report:
              precision    recall  f1-score   support

           A       0.95      0.99      0.97       232
           B       0.86      0.92      0.89       229
           C       0.95      0.91      0.93       201
           D       0.89      0.93      0.91       250
           E       0.90      0.95      0.92       238
           F       0.94      0.91      0.93       211
           G       0.96      0.93      0.94       230
           H       0.91      0.78      0.84       218
           I       0.97      0.90      0.94       221
           J       0.92      0.92      0.92       228
           K       0.93      0.90      0.92       188
           L       0.96      0.95      0.95       231
           M       0.98      0.97      0.97       252
           N       0.96      0.92      0.94       231
           O       0.91      0.93      0.92       218
           P       0.93      0.92      0.92       248
           Q       0.95      0.94      0.94       253
    